In [27]:
import gpxpy
import json

import pandas as pd
from geopy.distance import geodesic

In [28]:
def get_gpx_df(gpx_file):
    with open(gpx_file, 'r') as file:
        gpx = gpxpy.parse(file)

    data = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                data.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                })

    df = pd.DataFrame(data)

    df['prev_latitude'] = df['latitude'].shift(1)
    df['prev_longitude'] = df['longitude'].shift(1)
    
    df['distance'] = df.apply(lambda x: geodesic((x['prev_latitude'], x['prev_longitude']), (x['latitude'], x['longitude'])).meters if not pd.isna(x['prev_latitude']) and not pd.isna(x['prev_longitude']) else 0, axis=1)
    df['accumulated_distance'] = df['distance'].cumsum()

    df = df.drop(columns=[
        'prev_latitude', 
        'prev_longitude',
        'distance'
    ])
    
    df.rename(columns={'accumulated_distance': 'distance'}, inplace=True)
    
    return df

In [29]:
df = get_gpx_df('../public/track.gpx')
df.head(3)

,latitude,longitude,distance
0,1.34228,103.83519,0.000000
1,1.34265,103.83574,73.616069
2,1.34276,103.83592,97.054093


In [36]:
def get_segment_point(segment):
    mid = len(segment) // 2
    return segment[mid][0], segment[mid][1]

segments, current_segment = [], []
distance, distance_counter = 0, 0
sampling_interval_meters = 400.0

for i, row in df.iterrows():
    current_segment.append((row['latitude'], row['longitude']))
    current_distance_count = row['distance'] // sampling_interval_meters
    
    if current_distance_count > distance_counter:
        distance = round(row['distance'] / 1000.0, 2)
        segments.append((current_segment, distance))
        
        current_segment = []
        distance_counter = current_distance_count

if current_segment:
    segments.append((current_segment, distance))

points = [(get_segment_point(segment[0]), segment[1]) for segment in segments]

data = [{"name": f"{distance}km", "lat": point[0], "lon": point[1]} for (point, distance) in points]    
    
with open('../tests/data.json', 'w') as f:
    json.dump(data, f, indent=4)